In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 50
page_size = 100

reviews = []
ratings = []

for i in range(1, pages + 1):
    print(f"Scraping page {i}")

    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"

    # Collect HTML data from this page
    response = requests.get(url)

    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')

    # Find all <div> elements with the itemprop attribute set to "reviewRating"
    rating_elements = parsed_content.find_all("div", {"itemprop": "reviewRating"})

    for rating_element in rating_elements:
        # Find the <span> element with the itemprop attribute set to "ratingValue"
        rating_value_element = rating_element.find("span", {"itemprop": "ratingValue"})

        # Extract the text content of the element, which is the rating value
        rating = rating_value_element.text

        ratings.append(rating)
    
    print(f"   ---> {len(ratings)} total ratings")

    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    
    print(f"   ---> {len(reviews)} total reviews")

if len(reviews) != len(ratings):
    difference = len(reviews) - len(ratings)
    for i in range(0, difference):
        ratings.append(0)

# Store the ratings and reviews in a pandas DataFrame
df = pd.DataFrame({"rating": ratings, "review": reviews})

# Write the DataFrame to a CSV file
df.to_csv("ab-aviation.csv", index=False)
